In [ ]:
import requests
import re
from bs4 import BeautifulSoup
from IPython.core.display import HTML
import pandas as pd
import math
import datetime
req = requests.get("https://store.steampowered.com/search/?specials=1&page=1")
page = req.text
soup = BeautifulSoup(page, 'html.parser')
soup

##Laddar in beautifulsoup, importerar en massa paket som pandas å math, sparar steam sidan till req variabeln

## Getting title

In [ ]:
#Title getter
steamTitle = []


title = soup.findAll("span",{"class","title"})#Söker efter titeln på steams page
for span in title: #loopar igenom alla span obejct i title
    steamTitle.append(str(span.text)) #stoppar alla titles som den hittar in i steamTitle list
print(steamTitle)  



## Getting rating and converting to numbers

In [ ]:
#rating getter
ratingList = []

review_summary = soup.findAll('span', {"class":"search_review_summary"}) #Söker efter review rating
review_summary_str = str(review_summary) #konverterar review rating till en string
for element in review_summary:#loopar igenom alla element i review_summary
    reviewResult = element.attrs['data-tooltip-html'].split('<br>')
    #print(element.attrs['data-tooltip-html'].split('<br>'))

    if reviewResult[0] == 'Overwhelmingly Positive':#om reviewresult listan's current index motsvarar följande raitng
        ratingList.append("8 ")                     #>Konverterar till en string siffra som motsvarar ratingen
    elif reviewResult[0] == 'Very Positive':
        ratingList.append("7 ")
    elif reviewResult[0] == 'Positive':
         ratingList.append("6 ")
    elif reviewResult[0] == 'Mostly Positive':
         ratingList.append("5 ")
    elif reviewResult[0] == 'Mixed':
         ratingList.append("4 ")
    elif reviewResult[0] == 'Mostly Negative':
         ratingList.append("3 ")
    elif reviewResult[0] == 'Negative':
         ratingList.append("2 ")
    elif reviewResult[0] == 'Very Negative' or 'Overwhelmingly Neative':
         ratingList.append("1 ")

    else:
        ratingList.append("No score ")
print(ratingList)

## Getting review numbers

In [ ]:
#Review summary getter
reviewList = []


review_summary = soup.findAll('span', {"class":"search_review_summary"})#Söker efter search_review_summary i alla span
review_summary_str = str(review_summary) #konverterar det den hittar till en string 

for element in review_summary: 

    reviewResult = element.attrs['data-tooltip-html'].split('<br>')#
    #print(element.attrs['data-tooltip-html'].split('<br>'))
    reviewSplit = reviewResult[1].split(" ") #Splittar bort "Very positive"
    reviewList.append(reviewSplit[3]) #Sätter in 3e indexen (som är antalet reviews) in i listan
print(reviewList)


## Getting new price and discounted price

In [ ]:
#Discounted price
priceList = []
newPrice = []


price = soup.findAll("div",{"class","search_price"})#Söker efter pris i alla divs som har search_price klassen
for div in price:
    splitPrice = str(div.text).split('€') #tar bort € från string
    priceList.append(re.sub(' +', ' ', splitPrice[0]).strip("\r\n"))# re.sub(' +', ' ', splitPrice[0]).strip("\r\n") ->
    # -> replacerar alla spaces som skapas i free to play


    try:
      newPrice.append(re.sub(' +', ' ', splitPrice[1]).strip("\r\n")) #[1] current price???

    except:
      newPrice.append(re.sub(' +', ' ', splitPrice[0]).strip("\r\n"))

print(newPrice)

## Getting date for each game

In [ ]:
#releasedate getter
dateList = []

releaseDate = soup.findAll("div",{"class","search_released"})#Söker efter release date i alla div som har search_released
for div in releaseDate:
    dateList.append(str(div.text))#sätter in i listan och konverterar till en string
print(dateList)  

## Getting discount percentage

In [ ]:
## percantage getter
percentageList = []

releaseDate = soup.findAll("div",{"class","search_discount"})#Söker efter discount i alla divvar som har search_discount classes
for div in releaseDate:

    if not div.text.strip():#Om strängen inte har något i sig
        percentageList.append("No Discount")#Sätter in "No discount" om man inte hittar en discount
    else:#Oh strängen har något i sig
        percentageList.append(str(div.text).strip("\n"))#sätter in percentage

print(percentageList) 

## Getting platform avaliability

In [ ]:
platformListWin = []
platformListMac = []
platformListLinux = []

for platform in soup.find_all('div', {"class":"col search_name ellipsis"}):#Söker efter platformer i alla divar som har col search_name ellipsis klassen
    title = platform.find('span',attrs={'class':'title'}).text
    if platform.find('span',{"class":"win"}):
        platformListWin.append(1)
    else:
        platformListWin.append(0)

    if platform.find("span",{"class":"mac"}):#Om den hittar mac (bläää)
        platformListMac.append(1)
    else:
        platformListMac.append(0)

    if platform.find("span",{"class":"linux"}):#Om den hittar linux
        platformListLinux.append(1)
    else:
        platformListLinux.append(0)
print(platformListWin)

##  Getting current date when data was imported

In [ ]:
# when imported getter
currentDate = datetime.date.today().strftime("%d%b%Y")# tar current date och sätter det in i ..."currentDate"
currentDate

print(len(steamTitle),len(reviewList),len(ratingList),len( percentageList), len(dateList), len(newPrice),len(priceList),len(platformListWin),len(platformListLinux), len(platformListMac),len(currentDate ))

## Creating Dataframe and exporting to csv

In [ ]:

dataRows = None

dataRows = pd.DataFrame({"Game title":steamTitle, "Reviews":reviewList,"Rating":ratingList,
                                    "Sale percentage":percentageList,
                                        'Release date':dateList, 'Pris':newPrice,
                                           'OrdinariePris':priceList,'Win':platformListWin,
                                    'Linux':platformListLinux,
                                    'OSX':platformListMac,
                                    'Tid':currentDate}) 

dataRows.to_csv('steamsale.csv')
#dataRows.head(25)

dataRows.head(25)

## End of program